In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from pathlib import Path
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
sns.set_style("whitegrid")

In [2]:
aggregate=pd.read_csv('./aggregate.csv',engine='python')
aggregate.set_index('GEO_ID',inplace=True)
agg_train, agg_test = train_test_split(aggregate,test_size=0.2,shuffle=True,random_state=121)

In [ ]:
agg_train['unemp_bac']=agg_train['unemp_rate']*agg_train['bac_deg']
agg_train['bac2']=agg_train['bac_deg']**2
agg_train['unemp_rate2']=agg_train['unemp_rate']**2
agg_train['log_pop']=np.log10(agg_train['total_pop'])
agg_train['young_unemp']=np.log10(agg_train['pop_18_30_pc']*agg_train['unemp_rate'])
agg_train['log_pov']=np.log10(agg_train['pov_pc'])
agg_train['pov_bac']=agg_train['pov_pc']*agg_train['bac_deg']
agg_train['log_urban_pop']=np.log10(agg_train['urban']*agg_train['total_pop'])

In [ ]:
features=['white_pc','log_pop','']